## Data Preparation

Imports

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
df_raw = pd.read_csv("influenza.csv")
df_raw.head(15)

,Season,Region,County,CDC Week,Week Ending Date,Disease,Count,County Centroid,FIPS
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,INFLUENZA_UNSPECIFIED,0,"(43.7863965, -75.4426165)",36049
1,2014-2015,WESTERN,MONROE,53,01/03/2015,INFLUENZA_B,2,"(43.174966, -77.639663)",36055
2,2015-2016,METRO,NASSAU,52,01/02/2016,INFLUENZA_B,2,"(40.7506561, -73.582905)",36059
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,INFLUENZA_B,0,"(44.1089711, -73.7775726)",36031
4,2013-2014,METRO,ULSTER,41,10/12/2013,INFLUENZA_B,1,"(41.9472321, -74.265447)",36111
5,2016-2017,CAPITAL DISTRICT,HAMILTON,51,12/24/2016,INFLUENZA_B,0,"(43.6578786, -74.5024556)",36041
6,2017-2018,METRO,NASSAU,49,12/09/2017,INFLUENZA_A,16,"(40.7506561, -73.582905)",36059
7,2017-2018,CAPITAL DISTRICT,WARREN,1,01/06/2018,INFLUENZA_A,17,"(43.5551053, -73.8381388)",36113
8,2016-2017,WESTERN,STEUBEN,13,04/01/2017,INFLUENZA_A,7,"(42.2667252, -77.3855253)",36101
9,2018-2019,CAPITAL DISTRICT,SCHENECTADY,10,03/09/2019,INFLUENZA_A,98,"(42.8175421, -74.0435834)",36093


Add year column to a new dataframe created as df_year

In [0]:
df_year = df_raw.copy()
df_year['year'] = df_raw['Week Ending Date']
df_year['year'] = df_year['year'].str.slice(start = 6)
df_year.head()

,Season,Region,County,CDC Week,Week Ending Date,Disease,Count,County Centroid,FIPS,year
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,INFLUENZA_UNSPECIFIED,0,"(43.7863965, -75.4426165)",36049,2016
1,2014-2015,WESTERN,MONROE,53,01/03/2015,INFLUENZA_B,2,"(43.174966, -77.639663)",36055,2015
2,2015-2016,METRO,NASSAU,52,01/02/2016,INFLUENZA_B,2,"(40.7506561, -73.582905)",36059,2016
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,INFLUENZA_B,0,"(44.1089711, -73.7775726)",36031,2019
4,2013-2014,METRO,ULSTER,41,10/12/2013,INFLUENZA_B,1,"(41.9472321, -74.265447)",36111,2013


Separating disease column to different Influenza types based on their count

In [0]:
#One hot encoding Disease column

df_count = pd.get_dummies(df_year, columns=['Disease'])
#df_count.shape
df_count.head(15)

,Season,Region,County,CDC Week,Week Ending Date,Count,County Centroid,FIPS,year,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,0,"(43.7863965, -75.4426165)",36049,2016,0,0,1
1,2014-2015,WESTERN,MONROE,53,01/03/2015,2,"(43.174966, -77.639663)",36055,2015,0,1,0
2,2015-2016,METRO,NASSAU,52,01/02/2016,2,"(40.7506561, -73.582905)",36059,2016,0,1,0
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,0,"(44.1089711, -73.7775726)",36031,2019,0,1,0
4,2013-2014,METRO,ULSTER,41,10/12/2013,1,"(41.9472321, -74.265447)",36111,2013,0,1,0
5,2016-2017,CAPITAL DISTRICT,HAMILTON,51,12/24/2016,0,"(43.6578786, -74.5024556)",36041,2016,0,1,0
6,2017-2018,METRO,NASSAU,49,12/09/2017,16,"(40.7506561, -73.582905)",36059,2017,1,0,0
7,2017-2018,CAPITAL DISTRICT,WARREN,1,01/06/2018,17,"(43.5551053, -73.8381388)",36113,2018,1,0,0
8,2016-2017,WESTERN,STEUBEN,13,04/01/2017,7,"(42.2667252, -77.3855253)",36101,2017,1,0,0
9,2018-2019,CAPITAL DISTRICT,SCHENECTADY,10,03/09/2019,98,"(42.8175421, -74.0435834)",36093,2019,1,0,0


In [0]:
#Multiplying disease counts with new diseases columns (A, B, Unspecified)

df_count[["Disease_INFLUENZA_A", "Disease_INFLUENZA_B", "Disease_INFLUENZA_UNSPECIFIED"]] = df_count[["Disease_INFLUENZA_A", "Disease_INFLUENZA_B", "Disease_INFLUENZA_UNSPECIFIED"]].multiply(df["Count"], axis="index")
df_count.head(15)

,Season,Region,County,CDC Week,Week Ending Date,Count,County Centroid,FIPS,year,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,2016-2017,CENTRAL,LEWIS,42,10/22/2016,0,"(43.7863965, -75.4426165)",36049,2016,0,0,0
1,2014-2015,WESTERN,MONROE,53,01/03/2015,2,"(43.174966, -77.639663)",36055,2015,0,2,0
2,2015-2016,METRO,NASSAU,52,01/02/2016,2,"(40.7506561, -73.582905)",36059,2016,0,2,0
3,2018-2019,CAPITAL DISTRICT,ESSEX,20,05/18/2019,0,"(44.1089711, -73.7775726)",36031,2019,0,0,0
4,2013-2014,METRO,ULSTER,41,10/12/2013,1,"(41.9472321, -74.265447)",36111,2013,0,1,0
5,2016-2017,CAPITAL DISTRICT,HAMILTON,51,12/24/2016,0,"(43.6578786, -74.5024556)",36041,2016,0,0,0
6,2017-2018,METRO,NASSAU,49,12/09/2017,16,"(40.7506561, -73.582905)",36059,2017,16,0,0
7,2017-2018,CAPITAL DISTRICT,WARREN,1,01/06/2018,17,"(43.5551053, -73.8381388)",36113,2018,17,0,0
8,2016-2017,WESTERN,STEUBEN,13,04/01/2017,7,"(42.2667252, -77.3855253)",36101,2017,7,0,0
9,2018-2019,CAPITAL DISTRICT,SCHENECTADY,10,03/09/2019,98,"(42.8175421, -74.0435834)",36093,2019,98,0,0


In [0]:
#df_count['Week'] = pd.to_datetime(df_count.year.astype(str), format='%Y') + \
#             pd.to_timedelta(df_count['CDC Week'].mul(7).astype(str) + ' days')

In [0]:
#df_count = df_count.sort_values(by = 'Week')

In [0]:
# Sum different Influenza types
df_sum = df_count.groupby(['County','FIPS','County Centroid','year','CDC Week','Week Ending Date'])["Count","Disease_INFLUENZA_A","Disease_INFLUENZA_B","Disease_INFLUENZA_UNSPECIFIED"].sum().reset_index()
df_sum.head(15)

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0
1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0
2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0
3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0
4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0
5,ALBANY,36001,"(42.5882713, -73.9740136)",2009,45,11/14/2009,61,61,0,0
6,ALBANY,36001,"(42.5882713, -73.9740136)",2009,46,11/21/2009,29,29,0,0
7,ALBANY,36001,"(42.5882713, -73.9740136)",2009,47,11/28/2009,17,17,0,0
8,ALBANY,36001,"(42.5882713, -73.9740136)",2009,48,12/05/2009,8,8,0,0
9,ALBANY,36001,"(42.5882713, -73.9740136)",2009,49,12/12/2009,5,5,0,0


In [0]:
df_sum.to_csv('master.csv')

In [0]:
# Testing for above logic
df_sum.loc[(df_sum['County'] == 'KINGS') & (df_sum['Week Ending Date'] == '02/10/2018')]

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
7263,KINGS,36047,"(40.6782, -73.9442)",2018,6,02/10/2018,1404,919,473,12


Merge Unemployment dataset with master.csv

In [0]:
df_master = pd.read_csv("master.csv", index_col=0)
df_master.head()

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED
0,ALBANY,36001,"(42.5882713, -73.9740136)",2009,40,10/10/2009,4,4,0,0
1,ALBANY,36001,"(42.5882713, -73.9740136)",2009,41,10/17/2009,11,11,0,0
2,ALBANY,36001,"(42.5882713, -73.9740136)",2009,42,10/24/2009,27,27,0,0
3,ALBANY,36001,"(42.5882713, -73.9740136)",2009,43,10/31/2009,87,86,1,0
4,ALBANY,36001,"(42.5882713, -73.9740136)",2009,44,11/07/2009,77,76,1,0


In [0]:
df_unemp = pd.read_csv("unemployment.csv")
df_unemp.head()

,FIPS,State,County,UnempRate2009,UnempRate2018,UnempRate2017,UnempRate2016,UnempRate2015,UnempRate2014,UnempRate2010,UnempRate2007,PctEmpChange1018,PctEmpChange1718,PctEmpChange0718,PctEmpChange0710,PctEmpAgriculture,PctEmpMining,PctEmpConstruction,PctEmpManufacturing,PctEmpTrade,PctEmpTrans,PctEmpInformation,PctEmpFIRE,PctEmpServices,PctEmpGovt,NumCivEmployed,NumUnemployed2010,NumUnemployed2011,NumCivLaborForce2011,UnempRate2011,NumEmployed2011,NumEmployed2010,NumCivLaborForce2010,NumUnemployed2009,NumEmployed2009,NumCivLaborForce2009,UnempRate2008,NumCivLaborForce2012,NumEmployed2008,NumCivLaborForce2008,UnempRate2009.1,NumUnemployed2008,NumUnemployed2014,NumUnemployed2018,NumEmployed2018,NumCivLaborforce2018,NumUnemployed2017,NumEmployed2017,NumCivLaborforce2017,NumUnemployed2016,NumEmployed2016,NumCivLaborforce2016,NumCivLaborforce2015,NumEmployed2007,NumUnemployed2015,UnempRate2012,NumEmployed2014,NumCivLaborforce2014,UnempRate2013,NumUnemployed2013,NumEmployed2013,NumCivLaborforce2013,NumUnemployed2007,NumCivLaborforce2007,NumUnemployed2012,NumEmployed2012,NumEmployed2015
0,36000,NY,New York,8.3,4.1,4.7,4.9,5,6.3,8.6,4.6,4.69,0.74,1.02,-3.50,0.521535,0.071296,5.587058,6.231981,12.991835,5.236431,2.931230,8.050504,53.833530,4.544600,9467631,825639,789304,9517361,8.3,8728057,8769723,9595362,802999,8844486,9647485,5.4,9612232,9139080,9664773,8.3,525693,603720,393648,9181058,9574706,447799,9113601,9561400,463523,9093571,9557094,9561878,9088207,504408,8.5,8925698,9529418,7.7,745382,8913788,9659170,433849,9522056,818847,8793385,9057470
1,36001,NY,Albany,6.8,3.7,4.2,4.1,4,4.9,7.0,3.9,1.81,1.15,0.28,-1.51,0.242827,0.066683,4.455810,5.201905,12.425689,3.555589,2.239543,7.495549,52.036663,12.279742,158961,11247,10959,158289,6.9,147330,150264,161511,10754,148183,158937,4.9,160657,152756,160552,6.8,7796,7694,5844,152991,158835,6650,151246,157896,6416,151867,158283,158427,152567,6884,7.1,149537,157231,6.1,9746,151059,160805,6145,158712,11397,149260,151543
2,36003,NY,Allegany,8.7,5.6,6.6,6.3,6,6.2,9.2,5.5,-16.61,-0.01,-17.01,-0.48,3.074707,0.640771,6.442480,15.572223,9.959269,4.301609,1.331214,2.409100,51.847804,4.420823,20132,2218,2112,23657,8.9,21545,22022,24240,2082,21816,23898,6.4,23691,22578,24123,8.7,1545,1451,1096,18365,19461,1300,18367,19667,1265,18768,20033,20335,22129,1321,8.4,21866,23317,7.5,1785,22014,23799,1276,23405,1994,21697,19014
3,36005,NY,Bronx,12.0,5.7,6.3,7.1,7,9.8,12.0,6.8,11.53,0.23,19.70,7.33,0.059947,0.013471,5.063837,3.528790,13.203156,7.151710,1.741494,6.427294,59.094935,3.715366,593858,69810,69718,584353,11.9,514635,512477,582287,64242,469624,533866,7.6,594943,484145,523746,12.0,39601,59124,34319,571545,605864,38255,570215,608470,43362,565436,608798,605820,477487,47125,12.4,544535,603659,11.8,71033,532428,603461,34992,512479,73956,520987,558695
4,36007,NY,Broome,8.2,4.9,5.6,5.4,6,6.6,8.7,4.4,-9.90,0.95,-13.04,-3.49,0.567733,0.204888,5.118755,10.384021,15.655011,4.252275,1.650547,4.498369,53.599267,4.069135,87365,8528,8148,94727,8.6,86579,89316,97844,7947,89553,97500,5.6,94012,92690,98160,8.2,5470,5846,4115,80477,84592,4690,79717,84407,4654,81119,85773,86750,92549,5173,8.7,82128,87974,7.8,7155,84779,91934,4309,96858,8177,85835,81577


In [0]:
df_unemp.shape
df_master['UnempRate'] = df_master.apply(lambda _: '-', axis=1)
try:
  df_unemp = df_unemp.drop([0])
except:
  print("Row 0 not present")

Already row 0 dropped


In [0]:
for county_name in df_master['County'].unique():
  for unemp_year in range(2009,2019):
    df_master['UnempRate'].loc[(df_master['year'] == unemp_year) & (df_master['County'] == county_name)] = float(df_unemp['UnempRate' + str(unemp_year)].iloc[np.where(df_unemp['County'] == county_name.title())])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
df_master.sample(n = 10)
#df_master.head(20)

,County,FIPS,County Centroid,year,CDC Week,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change
9569,NEW YORK,36061,"(40.7831, -73.9712)",2016,50,12/17/2016,34,28,4,2,4.6,-,-
2694,CHENANGO,36017,"(42.489732, -75.6049051)",2018,1,01/06/2018,10,9,1,0,4.7,-,-
17260,TIOGA,36107,"(42.1256, -76.3637)",2015,46,11/21/2015,0,0,0,0,5,1.56073e+06,10.8
9963,NIAGARA,36063,"(43.203001, -78.71083)",2018,41,10/13/2018,0,0,0,0,5.2,-,-
12892,QUEENS,36081,"(40.7282, -73.7949)",2015,46,11/21/2015,3,1,2,0,5,7.38429e+07,1.3
1600,CAYUGA,36011,"(43.0085456, -76.5745866)",2011,12,03/26/2011,6,2,4,0,8.1,-,-
12200,OTSEGO,36077,"(42.6297762, -75.028841)",2015,7,02/21/2015,9,7,2,0,5,1.92558e+06,1.3
7098,KINGS,36047,"(40.6782, -73.9442)",2013,7,02/16/2013,135,42,91,2,9.4,6.00627e+07,0.6
4075,DUTCHESS,36027,"(41.7550085, -73.7399512)",2013,4,01/26/2013,46,33,13,0,6.7,1.18176e+07,3.6
10090,ONEIDA,36065,"(43.2427273, -75.4342824)",2012,12,03/24/2012,18,18,0,0,8.2,9.04835e+06,-


In [0]:
df_master.to_csv("master_UnEmp.csv")

Merge GDP 

In [0]:
df_gdp = pd.read_csv("real_gdp.csv")
df_gdp.head(15)

df_gdp = df_gdp[df_gdp.IndustryName == "All Industries"]
df_gdp = df_gdp.drop(columns=["IndustryName","LineCode"])
df_gdp.head()

,FIPS,Countyname,Postal,2012,2013,2014,2015,2013.1,2014.1,2015.1
0,36001,Albany,NY,24791703.0,25479702.0,26405639.0,26254937.0,2.8,3.6,-0.6
4,36003,Allegany,NY,1016724.0,1054031.0,1100681.0,1072033.0,3.7,4.4,-2.6
8,36005,Bronx,NY,29926313.0,29067477.0,28878091.0,28787000.0,-2.9,-0.7,-0.3
12,36007,Broome,NY,8695765.0,8525008.0,8341316.0,7987631.0,-2.0,-2.2,-4.2
16,36009,Cattaraugus,NY,2377326.0,2430050.0,2385269.0,2378817.0,2.2,-1.8,-0.3


In [0]:
df_master['GDP'] = df_master.apply(lambda _: '-', axis=1)
for county_name in df_master['County'].unique():
  for unemp_year in range(2012,2016):
    df_master['GDP'].loc[(df_master['year'] == unemp_year) & (df_master['County'] == county_name)] = float(df_gdp[str(unemp_year)].iloc[np.where(df_gdp['Countyname'] == county_name.title())])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
df_master['GDP_change'] = df_master.apply(lambda _: '-', axis=1)
for county_name in df_master['County'].unique():
  for unemp_year in range(2013,2016):
    df_master['GDP_change'].loc[(df_master['year'] == unemp_year) & (df_master['County'] == county_name)] = float(df_gdp[str(unemp_year) + '.1'].iloc[np.where(df_gdp['Countyname'] == county_name.title())])

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
df_master.sample(n = 10)

In [0]:
df_master.to_csv("master_GDP.csv")

Generate Target

In [0]:
for i in range(0, len(df_master)-2):
    if df_master.loc[i,'County'] == df_master.loc[i+2,'County']:
        df_master.loc[i, 'Target'] = df_master.loc[i+2, 'Count']
df_master.head()

In [0]:
df_master.to_csv("master_target.csv")

Merge Vaccination Rates

In [0]:
df_vaccination = pd.read_csv('vaccination.csv')
NYC_Counties = ['NEW YORK', 'BRONX', 'QUEENS', 'KINGS', 'RICHMOND']
df_vax = pd.merge(df_master, df_vaccination,  how='left', left_on=['year','CDC Week'], right_on = ['Year','CDC_Week'])
for i in range(1,len(df_vax)):
    if df_vax.loc[i, 'County'] in NYC_Counties:
        df_vax.loc[i, 'VaxRate'] = df_vax.loc[i, 'Child_NYC']
    else:
        df_vax.loc[i, 'VaxRate'] = df_vax.loc[i, 'Child_NYS']
df_vax = df_vax.drop(['Child_NYC','Child_NYS','Year','Month','CDC_Week'], axis = 1 ) #drop extra columns from df_vaccination

In [0]:
df_vax.to_csv("master_vax.csv")

Weather forecast

In [16]:
df_master = pd.read_csv("master_12-08.csv", index_col=0)
df_master.sample(5)

,Unnamed: 0.1,County,year,CDC Week,FIPS,County Centroid,Week Ending Date,Count,Disease_INFLUENZA_A,Disease_INFLUENZA_B,Disease_INFLUENZA_UNSPECIFIED,UnempRate,GDP,GDP_change,Population,flu,fluSymptoms,fever,cough,soreThroat,Target,VaxRate,PRCP,SNOW,TAVG,TMAX,TMIN,SNOWDepth,Density
8207,8207,MADISON,2016,13,36053,"(42.9100255, -75.6635749)",04/02/2016,24,21,3,0,5.3,NaN,NaN,71291.0,0.011148,0.013802,0.491038,0.360719,0.392215,9.0,0.617,0.201429,1.400000,41.214286,53.428571,29.000000,1.500000,108.8678150387881
4941,4941,FRANKLIN,2011,50,36033,"(44.5943743, -74.3106698)",12/17/2011,0,0,0,0,9.1,NaN,NaN,51541.0,0.000180,0.000357,0.036018,0.076212,0.037180,0.0,NaN,0.093333,0.500000,30.071429,37.857143,22.285714,1.000000,31.637325672755846
13748,13748,ROCKLAND,2011,8,36087,"(41.1547851, -74.0247721)",02/26/2011,11,5,6,0,7.1,NaN,NaN,315439.0,0.003960,0.002975,0.561145,0.257400,0.265200,5.0,NaN,0.200000,1.500000,21.142857,34.142857,8.142857,1.750000,1817.568424085278
5250,5250,FULTON,2012,4,36035,"(43.1156093, -74.4236782)",01/28/2012,0,0,0,0,11.0,1326795.0,NaN,54853.0,0.001593,0.001539,0.223074,0.134862,0.170601,0.0,NaN,0.344286,0.428571,25.785714,32.428571,19.142857,7.714286,110.7090237552223
12707,12707,QUEENS,2010,8,36081,"(40.7282, -73.7949)",02/27/2010,10,8,2,0,8.6,NaN,NaN,2234745.0,0.001650,0.001912,0.229840,0.065340,0.125840,6.0,NaN,0.828000,1.916667,34.857143,39.142857,30.571429,4.166667,20591.034736939095


In [0]:
for i in range(0, len(df_master)-2):
    if df_master.loc[i,'County'] == df_master.loc[i+2,'County']:
        df_master.loc[i, 'F_PRCP'] = df_master.loc[i+2, 'PRCP']
        df_master.loc[i, 'F_SNOW'] = df_master.loc[i+2, 'SNOW']
        df_master.loc[i, 'F_TAVG'] = df_master.loc[i+2, 'TAVG']
        df_master.loc[i, 'SNOWDepth'] = df_master.loc[i+2, 'SNOWDepth']


In [0]:
df_master.to_csv("master_F_weather.csv", index = False)